# Fine Tuning Roberta for Sentiment Analysis





### Introduction

In this tutorial I will be fine tuning a roberta model for the **Sentiment Analysis** problem. 

#### Flow of the notebook

The notebook will be divided into seperate sections to provide a organized walk through for the process used. This process can be modified for individual use cases. The sections are:

1. [Importing Python Libraries and preparing the environment](#section01)
2. [Importing and Pre-Processing the domain data](#section02)
3. [Preparing the Dataset and Dataloader](#section03)
4. [Creating the Neural Network for Fine Tuning](#section04)
5. [Fine Tuning the Model](#section05)
6. [Validating the Model Performance](#section06)
7. [Saving the model and artifacts for Inference in Future](#section07)

#### Technical Details

This script leverages on multiple tools designed by other teams. Details of the tools used below. Please ensure that these elements are present in your setup to successfully implement this script.

 - Data: 
	 - I will be using the dataset available at [Kaggle Competition](https://www.kaggle.com/c/movie-review-sentiment-analysis-kernels-only)
	 - I will be referring only to the first csv file from the data dump: `train.tsv`

 - Language Model Used:
	 - The RoBERTa model was proposed in RoBERTa: A Robustly Optimized BERT Pretraining Approach by Yinhan Liu, Myle Ott, Naman Goyal, Jingfei Du, Mandar Joshi, Danqi Chen, Omer Levy, Mike Lewis, Luke Zettlemoyer, Veselin Stoyanov. It is based on Google’s BERT model released in 2018.
	 - [Blog-Post](https://ai.facebook.com/blog/roberta-an-optimized-method-for-pretraining-self-supervised-nlp-systems/)
	 - [Research Paper](https://arxiv.org/pdf/1907.11692)
     - [Documentation for python](https://huggingface.co/transformers/model_doc/roberta.html)


 - Hardware Requirements:
	 - Python 3.6 and above
	 - Pytorch, Transformers and All the stock Python ML Libraries
	 - GPU enabled setup 

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* tqdm
* sklearn
* Robert Model and Tokenizer

Followed by that we will preapre the device for CUDA execeution. This configuration is needed if you want to leverage on onboard GPU. 

In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import time
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from collections import defaultdict, Counter
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
# # Setting up the device for GPU usage

# from torch import cuda
# device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
# train = pd.read_csv('../input/ruatd-dataset/train.csv', delimiter=',')

In [4]:
# from sklearn import preprocessing

# le = preprocessing.LabelEncoder()
# le.fit(train.Class)
# train['Class'] = le.transform(train.Class)


In [5]:
# train.shape

In [6]:
# train.head()

In [7]:
# train['Class'].unique()

In [8]:
# train.describe()

In [9]:
# new_df = train[['Text', 'Class']]
# new_df

In [10]:
# test_data = pd.read_csv('../input/ruatd-dataset/val.csv', delimiter=',')
# test_data['Class'] = le.transform(test_data.Class)
# test_data = test_data[['Text', 'Class']]

In [11]:
# test_data2 = pd.read_csv('../input/ruatd-dataset/test.csv', delimiter=',')

In [12]:
# test_data2 = test_data2[['Text']]
# test_data2

<a id='section03'></a>
### Preparing the Dataset and Dataloader

I will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. I will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing. 
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *SentimentData* Dataset Class
- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the Roberta model for training. 
- I am using the Roberta tokenizer to tokenize the data in the `TITLE` column of the dataframe. 
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/roberta.html#robertatokenizer)
- `target` is the encoded category on the news headline. 
- The *SentimentData* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training. 

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [13]:
# # Defining some key variables that will be used later on in the training
# MAX_LEN = 256
# TRAIN_BATCH_SIZE = 8
# VALID_BATCH_SIZE = 4
# # EPOCHS = 1
# LEARNING_RATE = 1e-05
# tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence", truncation=True, do_lower_case=True)

In [14]:
# class SentimentData(Dataset):
#     def __init__(self, dataframe, tokenizer, max_len):
#         self.tokenizer = tokenizer
#         self.data = dataframe
#         self.text = dataframe.Text
#         self.targets = self.data.Class
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.text)

#     def __getitem__(self, index):
#         text = str(self.text[index])
#         text = " ".join(text.split())

#         inputs = self.tokenizer.encode_plus(
#             text,
#             None,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             pad_to_max_length=True,
#             return_token_type_ids=True
#         )
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']
#         token_type_ids = inputs["token_type_ids"]


#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
#             'targets': torch.tensor(self.targets[index], dtype=torch.float)
#         }

In [15]:
# train_data = new_df
# test_data = test_data
# print("FULL Dataset: {}".format(new_df.shape))
# print("TRAIN Dataset: {}".format(train_data.shape))
# print("TEST Dataset: {}".format(test_data.shape))

# training_set = SentimentData(train_data, tokenizer, MAX_LEN)
# testing_set = SentimentData(test_data, tokenizer, MAX_LEN)
# # testing_set2 = SentimentData(test_data2, tokenizer, MAX_LEN)

In [16]:
# train_params = {'batch_size': TRAIN_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': 0
#                 }

# test_params = {'batch_size': VALID_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': 0
#                 }
# test2_params = {'batch_size': VALID_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': 0
#                 }
# training_loader = DataLoader(training_set, **train_params)
# testing_loader = DataLoader(testing_set, **test_params)
# # testing_loader2 = DataLoader(testing_set2, **test_params2)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `RobertaClass`. 
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs. 
 - The data will be fed to the Roberta Language model as defined in the dataset. 
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction. 
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 
 
#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output. 
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [17]:
# def collate_fn(input_data):
#     texts, labels = zip(*input_data)
#     labels = torch.LongTensor(labels)
#     inputs = tokenizer(texts, return_tensors='pt', padding='longest', max_length=256, truncation=True)
#     inputs['Class'] = labels
#     return inputs


# class Metric: # metric class for storing metrics (accuracy, loss)
#     def __init__(self):
#         self.storage = defaultdict(list)
    
#     def store(self, **kwargs):
#         for key in kwargs:
#             self.storage[key].append(kwargs[key])
            
#     def reset(self):
#         self.storage.clear()
        
#     def log(self):
#         for key in self.storage:
#             self.storage[key] = np.mean(self.storage[key])
#         return self.storage.items()

In [18]:
# class RobertaClass(torch.nn.Module):
#     def __init__(self):
#         super(RobertaClass, self).__init__()
#         self.l1 = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
#         self.pre_classifier = torch.nn.Linear(768, 2)
# #         self.dropout = torch.nn.Dropout(0.5)
# #        self.classifier1 = torch.nn.Linear(768, 100)
# #        self.classifier2 = torch.nn.Linear(100, 2)

#     def forward(self, input_ids, attention_mask, token_type_ids):
#         output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         hidden_state = output_1[0]
#         pooler = hidden_state[:, 0]
#         output = self.pre_classifier(pooler)
#         #pooler = torch.nn.ReLU()(pooler)
#         #pooler = self.dropout(pooler)
#         #output = self.classifier1(pooler)
#         #output = self.classifier2(output)
#         return output

In [19]:
# model = RobertaClass()
# model.to(device)

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process. 

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network. 

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size. 
- Subsequent output from the model and the actual category are compared to calculate the loss. 
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a loss of 0.8141926634122427.

In [20]:
# # Creating the loss function and optimizer
# loss_function = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-5)

In [21]:
# def calcuate_accuracy(preds, targets):
#     n_correct = (preds==targets).sum().item()
#     return n_correct

In [22]:
# for _,data in tqdm(enumerate(training_loader, 0)):
#     print(data)
#     break

In [23]:
# # Defining the training function on the 80% of the dataset for tuning the distilbert model

# def train(epoch):
#     tr_loss = 0
#     n_correct = 0
#     nb_tr_steps = 0
#     nb_tr_examples = 0
#     model.train()
#     for _,data in tqdm(enumerate(training_loader, 0)):
#         ids = data['ids'].to(device, dtype = torch.long)
#         mask = data['mask'].to(device, dtype = torch.long)
#         token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#         targets = data['targets'].to(device, dtype = torch.long)

#         outputs = model(ids, mask, token_type_ids)
#         loss = loss_function(outputs, targets)
#         tr_loss += loss.item()
#         big_val, big_idx = torch.max(outputs.data, dim=1)
#         n_correct += calcuate_accuracy(big_idx, targets)

#         nb_tr_steps += 1
#         nb_tr_examples+=targets.size(0)
        
#         if _%500==0:
#             loss_step = tr_loss/nb_tr_steps
#             accu_step = (n_correct*100)/nb_tr_examples 
#             print(f"Training Loss per 500 steps: {loss_step}")
#             print(f"Training Accuracy per 500 steps: {accu_step}")
#         optimizer.zero_grad()
#         loss.backward()
#         # # When using GPU
#         optimizer.step()

#     print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
#     epoch_loss = tr_loss/nb_tr_steps
#     epoch_accu = (n_correct*100)/nb_tr_examples
#     print(f"Training Loss Epoch: {epoch_loss}")
#     print(f"Training Accuracy Epoch: {epoch_accu}")

#     return 

In [24]:
# EPOCHS = 3
# try:
#     for epoch in range(EPOCHS):
#         train(epoch)
# except RuntimeError:
#     cuda.empty_cache()

<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data. 

This unseen data is the 20% of `train.tsv` which was seperated during the Dataset creation stage. 
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model. 

As you can see the model is predicting the correct category of a given sample to a 69.47% accuracy which can further be improved by training more.

In [25]:
# def valid(model, testing_loader):
#     model.eval()
#     n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
#     with torch.no_grad():
#         for _, data in tqdm(enumerate(testing_loader, 0)):
#             ids = data['ids'].to(device, dtype = torch.long)
#             mask = data['mask'].to(device, dtype = torch.long)
#             token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
#             targets = data['targets'].to(device, dtype = torch.long)
#             outputs = model(ids, mask, token_type_ids).squeeze()
#             loss = loss_function(outputs, targets)
#             tr_loss += loss.item()
#             big_val, big_idx = torch.max(outputs.data, dim=1)
#             n_correct += calcuate_accuracy(big_idx, targets)

#             nb_tr_steps += 1
#             nb_tr_examples+=targets.size(0)
            
#             if _%5000==0:
#                 loss_step = tr_loss/nb_tr_steps
#                 accu_step = (n_correct*100)/nb_tr_examples
#                 print(f"Validation Loss per 100 steps: {loss_step}")
#                 print(f"Validation Accuracy per 100 steps: {accu_step}")
#     epoch_loss = tr_loss/nb_tr_steps
#     epoch_accu = (n_correct*100)/nb_tr_examples
#     print(f"Validation Loss Epoch: {epoch_loss}")
#     print(f"Validation Accuracy Epoch: {epoch_accu}")
    
#     return epoch_accu


In [26]:
# acc = valid(model, testing_loader)
# print("Accuracy on test data = %0.2f%%" % acc)

<a id='section07'></a>
### Saving the Trained Model Artifacts for inference

This is the final step in the process of fine tuning the model. 

The model and its vocabulary are saved locally. These files are then used in the future to make inference on new inputs of news headlines.

In [27]:
# output_model_file = 'pytorch_roberta_sentiment.bin'
# output_vocab_file = './'

# model_to_save = model
# torch.save(model_to_save, output_model_file)
# tokenizer.save_vocabulary(output_vocab_file)

# print('All files saved')
# print('This tutorial is completed')

# GPT3

In [28]:
# import os
# from collections import defaultdict, Counter
# import torch
# import torch.nn.functional as F
# import pandas as pd
# import numpy as np
# from transformers import BertForSequenceClassification, BertTokenizer
# import pytorch_lightning as pl
# from tqdm.notebook import tqdm
# from sklearn.preprocessing import LabelEncoder
# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, GPT2LMHeadModel, AutoModelForSequenceClassification


# batch_size=1
# #model_name = 'DeepPavlov/rubert-base-cased-sentence'
# model_name = 'sberbank-ai/rugpt3medium_based_on_gpt2'
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
# val = pd.read_csv('val.csv')

# le = LabelEncoder() # закодируем лейблы 
# le.fit(train['Class'].values)

In [29]:
# def collate_fn(batch):
#     texts, labels = zip(*batch)
#     labels = torch.LongTensor(labels)
#     inputs = tokenizer(list(texts), return_tensors='pt', padding='max_length', max_length=256, truncation=True)
#     inputs['Class'] = labels
#     #inputs["Class"] = [
#     #    [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
#     #]
#     return inputs

# class TextDataset(torch.utils.data.Dataset):
#     def __init__(self, data, sort=False, le=None):
#         super().__init__()
#         self.texts = data['Text'].values
#         if 'Class' in data.columns: # если есть разметка
#             assert not data['Class'].isnull().any(), "Some labels are null"
#             if le is not None:
#                 self.labels = le.transform(data['Class'])
#             else:
#                 self.labels = data['Class'].values
        
#     def __len__(self):
#         return len(self.texts)
    
#     def __getitem__(self, idx):
#         if hasattr(self, 'labels'):
#             return self.texts[idx], self.labels[idx]
#         else:
#             return self.texts[idx], []

# train = TextDataset(train, le=le)
# val = TextDataset(val, le=le)
# test = TextDataset(test, le=le)

# train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
# val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
# test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [30]:
# class Metric: # metric class for storing metrics (accuracy, loss)
#     def __init__(self):
#         self.storage = defaultdict(list)
    
#     def store(self, **kwargs):
#         for key in kwargs:
#             self.storage[key].append(kwargs[key])
            
#     def reset(self):
#         self.storage.clear()
        
#     def log(self):
#         for key in self.storage:
#             self.storage[key] = np.mean(self.storage[key])
#         return self.storage.items()
        
# class BertClassifier(pl.LightningModule):
#     def __init__(self, model_name, lr=1e-5, num_labels=2):
#         super().__init__()
#         #self.bert = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
#         #self.bert = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
#         self.bert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, pad_token_id=tokenizer.eos_token_id)
#         self.metric = Metric()
#         self.learning_rate = lr
        
#     def configure_optimizers(self):
#         optimizer = torch.optim.AdamW(self.bert.parameters(), lr=self.learning_rate)
#         return optimizer
        
#     def forward(self, x):
#         #print(inputs["attention_mask"])
#         return self.bert(**x)
#     #
    
#     def training_step(self, batch, batch_idx):
#         labels = batch.pop('Class')
#         logits = self.bert(**batch).logits
#         loss = F.cross_entropy(logits, labels)
#         predictions = logits.argmax(axis=1)
#         accuracy = torch.mean((predictions == labels).double())
#         self.metric.store(loss=loss.item(), accuracy=accuracy.item())
#         if batch_idx % 100: # every 100 batches - log metrics (mean of last 100 batches)
#             for k,v in self.metric.log():
#                 self.log(f'train/{k}', v)
#             self.metric.reset()
#         return loss
    
#     def validation_step(self, batch, batch_idx):
#         #print(batch)
#         labels = batch.pop('Class')
#         logits = self.bert(**batch).logits
#         loss = F.cross_entropy(logits, labels)
#         self.log('val/loss', loss)
#         predictions = logits.argmax(axis=1)
#         self.log('val/accuracy', torch.mean((predictions == labels).double()))
        

# #tokenizer = BertTokenizer.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"
# model = BertClassifier(model_name)
# model.bert.resize_token_embeddings(len(tokenizer))

# #tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# version = f"{model_name}_binary"
# logger = pl.loggers.TensorBoardLogger(save_dir=os.getcwd(), name='lightning_logs', version=version)
# trainer = pl.Trainer(
#     logger=logger, 
#     gpus=[0],
#     max_epochs=3, 
#     num_sanity_val_steps=1
# )

# trainer.fit(model, train_loader, val_loader)

In [31]:
# from torch import nn
# class SimpleGPT2SequenceClassifier(nn.Module):
#     def __init__(self, hidden_size: int, num_classes:int ,max_seq_len:int, gpt_model_name:str):
#         super(SimpleGPT2SequenceClassifier,self).__init__()
#         self.gpt2model = AutoModelForCausalLM.from_pretrained(gpt_model_name)
#         self.fc1 = nn.Linear(hidden_size*max_seq_len, num_classes)
        
#     def forward(self, input_id, mask):
#         """
#         Args:
#                 input_id: encoded inputs ids of sent.
#         """
#         gpt_out, _ = self.gpt2model(input_ids=input_id, attention_mask=mask, return_dict=False)
#         batch_size = gpt_out.shape[0]
#         linear_output = self.fc1(gpt_out.view(batch_size,-1))
#         return linear_output
# THIS ONE WAS COMMENTED

In [32]:
# model = SimpleGPT2SequenceClassifier(hidden_size=1536, 
#                                      num_classes=2, max_seq_len=128, gpt_model_name=model_name).cuda()
# THIS ONE WAS COMMENTED

In [33]:
# #logger.info('Loading GPT2Tokenizer.')
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.padding_side = "left"
# tokenizer.pad_token = tokenizer.eos_token THIS ONE WAS COMMENTED

In [34]:
# model.cuda()
# def get_accuracy_and_pred(model, loader): # используйте эту функцию для получения accuracy и предсказаний
#     preds = []
#     model.eval()
#     labels = None
#     accs = 0
#     ns = 0
#     for batch in tqdm(loader):
#         for key in batch:
#             #print(key)
#             batch[key] = batch[key].to(model.device)
#         #print(batch)
#         labels = batch.pop('Class')
#         #print(labels)

#         with torch.no_grad():
#             pred = model(batch).logits.argmax(axis=1)
#         #print(pred)
#         #print(labels.size())
#         if labels.size()[1] > 0:
#             #print(labels)
#             accs += torch.sum((pred == labels).double())
#         preds.append(pred.cpu().numpy())
#         ns += len(pred)
        
#         #print(accs, ns)
#     return accs/ns, np.concatenate(preds)

# acc, preds = get_accuracy_and_pred(model, test_loader)

In [35]:
# np.save('test_preds_gpt3med256.npy', le.inverse_transform(preds))
# print(f"Test accuracy: {acc}")

In [36]:
# f = np.load('test_preds_gpt3med256.npy', allow_pickle=True)
# f

In [37]:
# df = pd.read_csv('sample_submit_binary.csv', index_col=False)
# df

In [38]:
# # Drop that column
# df.drop('Class', axis = 1, inplace = True)

# # Put whatever series you want in its place
# df['Class'] = f

In [39]:
# df.to_csv('submit_gpt3med256.csv', index=False)

In [40]:
# breaaaaa

# RUBERT

In [41]:
import os
from collections import defaultdict, Counter
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer
import pytorch_lightning as pl
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification


batch_size=14
model_name = 'DeepPavlov/rubert-base-cased-conversational'
#model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
train = pd.read_csv('../input/ruatd-dataset/train.csv')
test = pd.read_csv('../input/ruatd-dataset/test.csv')
val = pd.read_csv('../input/ruatd-dataset/val.csv')

le = LabelEncoder() # закодируем лейблы 
le.fit(train['Class'].values)

LabelEncoder()

In [42]:
tokenizer = BertTokenizer.from_pretrained(model_name)
#tokenizer.pad_token = tokenizer.eos_token
def collate_fn(input_data):
    texts, labels = zip(*input_data)
    labels = torch.LongTensor(labels)
    inputs = tokenizer(texts, return_tensors='pt', padding='longest', max_length=256, truncation=True)
    inputs['Class'] = labels
    return inputs

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, data, sort=False, le=None):
        super().__init__()
        self.texts = data['Text'].values
        if 'Class' in data.columns: # если есть разметка
            assert not data['Class'].isnull().any(), "Some labels are null"
            if le is not None:
                self.labels = le.transform(data['Class'])
            else:
                self.labels = data['Class'].values
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        if hasattr(self, 'labels'):
            return self.texts[idx], self.labels[idx]
        else:
            return self.texts[idx], []

class Metric: # metric class for storing metrics (accuracy, loss)
    def __init__(self):
        self.storage = defaultdict(list)
    
    def store(self, **kwargs):
        for key in kwargs:
            self.storage[key].append(kwargs[key])
            
    def reset(self):
        self.storage.clear()
        
    def log(self):
        for key in self.storage:
            self.storage[key] = np.mean(self.storage[key])
        return self.storage.items()
        
class BertClassifier(pl.LightningModule):
    def __init__(self, model_name, lr=1e-5, num_labels=2):
        super().__init__()
        #self.bert = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.bert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        #self.bert = AutoModelForCausalLM.from_pretrained(model_name, num_labels=num_labels)
        self.metric = Metric()
        self.learning_rate = lr
        
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.bert.parameters(), lr=self.learning_rate)
        return optimizer
        
    def forward(self, x):
        return self.bert(**x)
    
    def training_step(self, batch, batch_idx):
        labels = batch.pop('Class')
        logits = self.bert(**batch).logits
        loss = F.cross_entropy(logits, labels)
        predictions = logits.argmax(axis=1)
        accuracy = torch.mean((predictions == labels).double())
        self.metric.store(loss=loss.item(), accuracy=accuracy.item())
        if batch_idx % 100: # every 100 batches - log metrics (mean of last 100 batches)
            for k,v in self.metric.log():
                self.log(f'train/{k}', v)
            self.metric.reset()
        return loss
    
    def validation_step(self, batch, batch_idx):
        labels = batch.pop('Class')
        logits = self.bert(**batch).logits
        loss = F.cross_entropy(logits, labels)
        self.log('val/loss', loss)
        predictions = logits.argmax(axis=1)
        self.log('val/accuracy', torch.mean((predictions == labels).double()))

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

In [43]:
train = TextDataset(train, le=le)
val = TextDataset(val, le=le)
test = TextDataset(test, le=le)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

#tokenizer = BertTokenizer.from_pretrained(model_name).cuda()

model = BertClassifier(model_name, num_labels=len(le.classes_)).cuda()

#here is freezing layers
for name, param in model.named_parameters():
    if name.startswith("bert.encoder.layer.1"):
        param.requires_grad = False
    if name.startswith("bert.encoder.layer.2"):
        param.requires_grad = False
    if name.startswith("bert.encoder.layer.3"):
        param.requires_grad = False
    if name.startswith("bert.encoder.layer.4"):
        param.requires_grad = False

version = f"{model_name}_binary"
logger = pl.loggers.TensorBoardLogger(save_dir=os.getcwd(), name='lightning_logs', version=version)
trainer = pl.Trainer(
    logger=logger, 
    gpus=[0],
    max_epochs=5, 
    num_sanity_val_steps=1
)
trainer.fit(model, train_loader, val_loader)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassi

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [44]:
# for name, param in model.named_parameters():
#      print(name, param.requires_grad)

In [45]:
model.cuda()
def get_accuracy_and_pred(model, loader): # используйте эту функцию для получения accuracy и предсказаний
    preds = []
    model.eval()
    labels = None
    accs = 0
    ns = 0
    for batch in tqdm(loader):
        for key in batch:
            #print(key)
            batch[key] = batch[key].to(model.device)
        #print(batch)
        labels = batch.pop('Class')
        #print(labels)

        with torch.no_grad():
            pred = model(batch).logits.argmax(axis=1)
        #print(pred)
        #print(labels.size())
        if labels.size()[1] > 0:
            #print(labels)
            accs += torch.sum((pred == labels).double())
        preds.append(pred.cpu().numpy())
        ns += len(pred)
        
        #print(accs, ns)
    return accs/ns, np.concatenate(preds)

acc, preds = get_accuracy_and_pred(model, test_loader)

  0%|          | 0/4610 [00:00<?, ?it/s]

In [46]:
np.save('test_preds_rubert_based_case_conversational_5ep.npy', le.inverse_transform(preds))
print(f"Test accuracy: {acc}")

Test accuracy: 0.0


In [47]:
df_testt = pd.read_csv('../input/ruatd-dataset/test.csv', index_col=False)
df_testt.drop('Text', axis = 1, inplace = True)
df_testt

,Id
0,8
1,12
2,14
3,16
4,17
...,...
64528,215100
64529,215101
64530,215103
64531,215108


In [48]:
f = np.load('test_preds_rubert_based_case_conversational_5ep.npy', allow_pickle=True)

# df = pd.read_csv('sample_submit_binary.csv', index_col=False)
# df.drop('Class', axis = 1, inplace = True)

df_testt['Class'] = f

df_testt.to_csv('test_preds_rubert_based_case_conversational_5ep.csv', index=False)

# Ансамбль

In [49]:
# f512 = np.load('test_preds_rubert_based_case512_2ep.npy', allow_pickle=True)
# f256 = np.load('test_preds_rubert_based_case256_2ep.npy', allow_pickle=True)
# f128 = np.load('test_preds_gpt3med256.npy', allow_pickle=True)

In [50]:
# ans = []

# for i, x in enumerate(f128):
#     y = f256[i]
#     z = f512[i]
#     if x == y == z:
#         ans.append(x)
#     elif x == y != z:
#         ans.append(x)
#     elif x == z != y:
#         ans.append(x)
#     elif x != y == z:
#         ans.append(y)

In [51]:
# df = pd.read_csv('sample_submit_binary.csv', index_col=False)
# df.drop('Class', axis = 1, inplace = True)

# df['Class'] = ans
# df.to_csv('submit_ansgpt256based_case_256_512_2ep.csv', index=False)

In [52]:
# for i, x in enumerate(f128):
#     if x != ans[i]:
#         print(x)